In [501]:
import pandas as pd
import os
import numpy as np
import sqlite3

**Step 1**

In [504]:
#Lets load the scrapped data
all_scrabbed = pd.read_csv('C:\\Users\\amir\\OneDrive\\Desktop\\New Hiring lead process\\Using python\\All scrapped leads\\March 25 all scrapped 025.csv')
len(all_scrabbed)

8194

In [506]:
#We need to remove the duplicate companies from the data

all_scrabbed = all_scrabbed.drop_duplicates(subset='Company Name', keep='first')
all_scrabbed = all_scrabbed.dropna(subset = ['Company Name'])
len(all_scrabbed)

1915

In [508]:
all_scrabbed.columns

Index(['Unnamed: 0', 'Company Name', 'Job Posting', 'Location',
       'Job Post date', 'Job links', 'Unnamed: 6'],
      dtype='object')

In [510]:
#Lets remove all the Bad Job Posts scrapped unintentionally
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSUQsbmb3MlADPi5q92Eeh5W6nwa8sgszHtctytwl_y2nlUnkJOHj377DJzasaugC_z7yrDU0FKA9cC/pub?gid=0&single=true&output=csv"
badtitle = pd.read_csv(url)
bad_job_post_list = badtitle['Bad Job Title'].tolist()

all_scrabbed = all_scrabbed[~all_scrabbed['Job Posting'].isin(bad_job_post_list)]#deleting the created list from Job posts column
len(all_scrabbed)

1829

In [512]:
# 
# all_scrabbed.to_csv('C:\\Users\\amir\\OneDrive\\Desktop\\New Hiring lead process\\Using python\\All scrapped leads\\Cleaned scrapped data\\25 march scrabbed_leads_without_duplicate.csv')

**Step 2**

1. Make sure in the master apollo all the companies should have company linkedin otherwise it will create a issue while mapping Job posting, job link SD-Team_size

1.1 Sort the company name A-Z so that all the missing company linkedin links visible clearly 

2. Make sure job poting and job link does not have blanks or #NA vlaues


3. We also need to remove #n/a from job post and job link

4. We need to add companies simplified names at the end of the process.

**5. We must short the urls before uploading the data**

6. While Creating master sheet make sure to remove all the duplicates through the linkedin link, otherwise it will not match the SD-team size and other column correctly

7. We need to clean the active sequence

In [524]:
#Lets upload the apollo master sheet
dfapollo = pd.read_csv('C:\\Users\\amir\\OneDrive\\Desktop\\New Hiring lead process\\Using python\\Apollo master sheet\\From Apollo\\March 25 hiring 025 - All Apollo.csv')
len(dfapollo)

1087

**we already know that Apollo's data linkedin links and Cognism data linkedin link is not matched**

In [527]:
print(dfapollo['linkedin_url'][0]) # in the below linkedin we can see it has on http instead https
print(dfapollo['company_linkedin_url'][100])

http://www.linkedin.com/in/christopher-murray1986
http://www.linkedin.com/company/capstone-investment-advisors


In [529]:
#we have linkedin link start from http, we are correcting them with https so that this could become our primary key
dfapollo['linkedin_url'] = dfapollo['linkedin_url'].str.replace('http://','https://', regex = False) #replaced http with https
print(dfapollo['linkedin_url'][0])
dfapollo['company_linkedin_url'] = dfapollo['company_linkedin_url'].str.replace('http://','https://', regex = False) #replaced http with https
print(dfapollo['company_linkedin_url'][100])

https://www.linkedin.com/in/christopher-murray1986
https://www.linkedin.com/company/capstone-investment-advisors


In [531]:
dfapollo.columns

Index(['linkedin_url', 'full_name', 'first_name', 'last_name', 'email',
       'email validation', 'job_title', 'company', 'company_website', 'city',
       'state', 'country', 'industry', 'keywords', 'employees', 'company_city',
       'company_state', 'company_country', 'company_linkedin_url',
       'company_twitter_url', 'company_facebook_url', 'twitter_url',
       'facebook_url', 'company_about', 'Job Posting', 'Full Links',
       'Job Link', 'SD Team'],
      dtype='object')

In [533]:
#lets rename the linkedin column name as well
dfapollo = dfapollo.rename(columns={'linkedin_url': 'linkedin_link'})
dfapollo = dfapollo.rename(columns={'company_linkedin_url': 'company_linkedin_link'})
dfapollo = dfapollo.rename(columns={'SD Team': 'sd_team_size'})
dfapollo = dfapollo.rename(columns={'company_website':'website'})
dfapollo = dfapollo.rename(columns = {'Job Link':'job link'})
dfapollo = dfapollo.rename(columns={'Full Links': 'full links'})

In [535]:
#lets remove duplicates from linkedin link so that this could be treated as primary key
dfapollo = dfapollo.drop_duplicates(subset = 'linkedin_link', keep = 'first')
len(dfapollo)

1087

In [537]:
#below given columns are not usefull in apollo data lets delete them as well
apollo_col_need_to_delete =['company_twitter_url','company_facebook_url', 'twitter_url', 'facebook_url','city', 'state', 'country']
dfapollo.drop(columns=apollo_col_need_to_delete, inplace=True)


In [539]:
dfapollo.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name', 'email',
       'email validation', 'job_title', 'company', 'website', 'industry',
       'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about',
       'Job Posting', 'full links', 'job link', 'sd_team_size'],
      dtype='object')

In [541]:
print(len(dfapollo))
print(len(dfapollo.columns))

1087
21


**Step 3**

**Lets upload and perform cleaning on cognism sheet**

In [545]:
#lets upload cognism data
cognismdata = pd.read_csv('C:\\Users\\amir\\OneDrive\\Desktop\\New Hiring lead process\\Using python\\Cognism data\\March 25 Hiring leads 025.csv')
len(cognismdata)

1228

In [547]:
cognismdata.columns

Index(['Profile ID', 'First Name', 'Last Name', 'sId', 'List Name', 'Email',
       'Email Quality', 'Lead Source', 'Education', 'Personal Linkedin URL',
       'Company Name', 'Company Alias', 'Job Title', 'Country', 'City',
       'State', 'Mobile', 'Direct', 'Office', 'HQ', 'Website', 'Headcount',
       'Industries', 'Department', 'Company Address Line', 'Company City',
       'Company Post Code/ZIP', 'Company County/State', 'Company Country',
       'Company HQ Address Line', 'Company HQ City',
       'Company HQ Post Code/ZIP', 'Company HQ County/State',
       'Company HQ Country', 'Company Linkedin URL', 'Company Type',
       'Company Description', 'Technologies', 'Financials',
       'Company Founded Year', 'Seniority', 'Hiring Title 1', 'Hiring URL 1',
       'Hiring Location 1', 'Hiring Date 1', 'Hiring Title 2', 'Hiring URL 2',
       'Hiring Location 2', 'Hiring Date 2', 'Hiring Title 3', 'Hiring URL 3',
       'Hiring Location 3', 'Hiring Date 3', 'Hiring Title 4', 'Hiri

In [549]:
#Lets create a list of unwanted columns of cognism
cog_col_need_to_delete =[
'sId'
,'Lead Source'
,'Education'
,'Company Alias'
,'Direct'
,'Office'
,'HQ'
,'Department'
,'Company Address Line'
,'Company Post Code/ZIP'
,'Financials'
,'Company Founded Year'
,'Seniority'
,'Hiring Title 1'
,'Hiring URL 1'
,'Hiring Location 1'
,'Hiring Date 1'
,'Hiring Title 2'
,'Hiring URL 2'
,'Hiring Location 2'
,'Hiring Date 2'
,'Hiring Title 3'
,'Hiring URL 3'
,'Hiring Location 3'
,'Hiring Date 3'
,'Hiring Title 4'
,'Hiring URL 4'
,'Hiring Location 4'
,'Hiring Date 4'
,'Hiring Title 5'
,'Hiring URL 5'
,'Hiring Location 5'
,'Hiring Date 5'
,'Location Move - From Country'
,'Location Move - From State'
,'Location Move - To Country'
,'Location Move - To State'
,'Location Move Date'
,'Job Change - Previous Company'
,'Job Change - Previous Title'
,'Job Change - New Company'
,'Job Change - New Title'
,'Job Change Date'
,'Profile ID'
,'Company Type'
,'List Name'
 ,'Company HQ Address Line',
'Company HQ City', 
'Company HQ Post Code/ZIP',
'Company HQ County/State', 
'Company HQ Country', 'Country', 'City','State']
# ['SIC', 'ISIC', 'NAICS']

In [551]:
#Lets delete the created list from cognism data
cognismdata.drop(columns=cog_col_need_to_delete, inplace=True)

In [553]:
len(cognismdata.columns)

17

In [555]:
#lets rename the column names of the cognism sheet so that apollo coulumns and cognism column name match.
#creating a list of old column name and new column names for cognims data
column_mappings = [
    ('Personal Linkedin URL', 'linkedin_link'),
    ('First Name', 'first_name'),
    ('Last Name', 'last_name'),
    ('Job Title','job_title'),
    ('Company Name','company'),
    ('Industries','industry'),
    ('Technologies','keywords'),
    ('Headcount','employees'),
    ('Company County/State','company_state'),
    ('Company Country','company_country'),
    ('Company Linkedin URL','company_linkedin_link'),
    ('Company Description','company_about'),
    ('Email','email'),
    ('Company City','company_city'),
    ('Mobile','mobile'),
    ('Email Quality','email_validation')
]

dic_column_mapping = dict(column_mappings)
cognismdata.rename(columns=dic_column_mapping, inplace=True)


In [557]:
cognismdata.columns

Index(['first_name', 'last_name', 'email', 'email_validation', 'linkedin_link',
       'company', 'job_title', 'mobile', 'Website', 'employees', 'industry',
       'company_city', 'company_state', 'company_country',
       'company_linkedin_link', 'company_about', 'keywords'],
      dtype='object')

In [559]:
#we need to add FUll NAME coulmn
cognismdata['full_name'] = cognismdata['first_name'] + ' ' + cognismdata['last_name']
cognismdata.columns

Index(['first_name', 'last_name', 'email', 'email_validation', 'linkedin_link',
       'company', 'job_title', 'mobile', 'Website', 'employees', 'industry',
       'company_city', 'company_state', 'company_country',
       'company_linkedin_link', 'company_about', 'keywords', 'full_name'],
      dtype='object')

In [561]:
#Lets rearrange cognism column to verify what need to do next with apollo
cognismdata = cognismdata[['linkedin_link','full_name','first_name','last_name','email_validation','email','job_title','company','Website',
                          'industry','keywords','employees','company_city','company_state','company_country','company_linkedin_link','company_about',
                          'mobile']]

In [563]:
cognismdata.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name',
       'email_validation', 'email', 'job_title', 'company', 'Website',
       'industry', 'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about', 'mobile'],
      dtype='object')

In [565]:
dfapollo.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name', 'email',
       'email validation', 'job_title', 'company', 'website', 'industry',
       'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about',
       'Job Posting', 'full links', 'job link', 'sd_team_size'],
      dtype='object')

In [567]:
print(len(dfapollo.columns))
print(len(cognismdata.columns))

21
18


In [569]:
# Strip leading/trailing spaces and convert to lowercase
dfapollo.columns = dfapollo.columns.str.strip().str.lower()
cognismdata.columns = cognismdata.columns.str.strip().str.lower()

In [571]:
cognismdata.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name',
       'email_validation', 'email', 'job_title', 'company', 'website',
       'industry', 'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about', 'mobile'],
      dtype='object')

Now we need to add mobile column to the dfapollo and need to add 'job postin','job link','SD_team_size','Extracognism' to the cognism data

**Lets simultaneously add 'job postin','job link','sd_team size','extracognism' to the cognism data**

In [575]:
#Lets first map 'Job Posting', 'full links', 'job link', 'sd_team_size' to cognismdata
lookup_dicts = {
    'sd_team_size':dfapollo.set_index('company_linkedin_link')['sd_team_size'].to_dict(),
    'job posting':dfapollo.set_index('company_linkedin_link')['job posting'].to_dict(),
    'job link':dfapollo.set_index('company_linkedin_link')['job link'].to_dict(),
    'full links':dfapollo.set_index('company_linkedin_link')['full links'].to_dict()
}
#lets map each column to cognism data
for column_name, x in lookup_dicts.items():
    cognismdata[column_name] = cognismdata['company_linkedin_link'].map(x)


In [577]:
cognismdata.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name',
       'email_validation', 'email', 'job_title', 'company', 'website',
       'industry', 'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about', 'mobile',
       'sd_team_size', 'job posting', 'job link', 'full links'],
      dtype='object')

In [579]:
# Create a lookup dictionary from dfapollo for cognismdata so that we can have equal columns.
# lookup_dict = dfapollo.set_index('company_linkedin_link')['sd_team_size'].to_dict()#Mapping and adding sd_team size col to cognims
# # Add a new 'tag' column to dfapollo using the lookup dictionary
# cognismdata['sd_team_size'] = dfapollo['company_linkedin_link'].map(lookup_dict)
# print(cognismdata['sd_team_size'])

In [581]:
# #Adding job posting to cognism data
# lookup_dict = dfapollo.set_index('company_linkedin_link')['job posting'].to_dict()#Mapping and adding sd_team size col to cognims
# # Add a new 'tag' column to dfapollo using the lookup dictionary
# cognismdata['job posting'] = dfapollo['company_linkedin_link'].map(lookup_dict)
# print(cognismdata['job posting'])

In [583]:
#adding job link to cognism data
# lookup_dict = dfapollo.set_index('company_linkedin_link')['job link'].to_dict()#Mapping and adding sd_team size col to cognims
# # Add a new 'tag' column to dfapollo using the lookup dictionary
# cognismdata['job link'] = dfapollo['company_linkedin_link'].map(lookup_dict)
# print(cognismdata['job link'])

In [585]:
print(len(cognismdata.columns))
print(len(cognismdata))

22
1228


In [587]:
cognismdata.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name',
       'email_validation', 'email', 'job_title', 'company', 'website',
       'industry', 'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about', 'mobile',
       'sd_team_size', 'job posting', 'job link', 'full links'],
      dtype='object')

**lets again do some cleaning and filtering on cognism data so that we can add mobile number to apollo data**

In [590]:
#Lets check and remove the duplicates from cognism sheet
cognismdata = cognismdata.drop_duplicates(subset = 'linkedin_link', keep = 'first')
len(cognismdata)

1215

In [592]:
#We need to remove all the row having no mobile numbers from cognism data
cognismdata = cognismdata[cognismdata['mobile'].notna() & (cognismdata['mobile'] != '')]#removing contacts with no mobile numbers
len(cognismdata)

1209

In [594]:
# Ensure the column is of type string
cognismdata['mobile'] = cognismdata['mobile'].astype(str)
# # Remove leading/trailing whitespace
cognismdata['mobile'] = cognismdata['mobile'].str.strip()

cog_mobile_filter = cognismdata[cognismdata['mobile'].str.startswith('+44 7')]
len(cog_mobile_filter)

716

**Step 3**

**We need to map mobile numbers to the apollo data from final_mobile_cog dataset**

**Step 4**

In [599]:
print("Columns in DataFrame:", dfapollo.columns.tolist())

Columns in DataFrame: ['linkedin_link', 'full_name', 'first_name', 'last_name', 'email', 'email validation', 'job_title', 'company', 'website', 'industry', 'keywords', 'employees', 'company_city', 'company_state', 'company_country', 'company_linkedin_link', 'company_about', 'job posting', 'full links', 'job link', 'sd_team_size']


In [601]:
print("Columns in DataFrame:", cog_mobile_filter.columns.tolist())

Columns in DataFrame: ['linkedin_link', 'full_name', 'first_name', 'last_name', 'email_validation', 'email', 'job_title', 'company', 'website', 'industry', 'keywords', 'employees', 'company_city', 'company_state', 'company_country', 'company_linkedin_link', 'company_about', 'mobile', 'sd_team_size', 'job posting', 'job link', 'full links']


In [603]:
# Create a lookup dictionary from cog_mobile_filter
lookup_dict = cog_mobile_filter.set_index('linkedin_link')['mobile'].to_dict()
# Add a new 'mobile' column to dfapollo using the lookup dictionary
dfapollo['mobile'] = dfapollo['linkedin_link'].map(lookup_dict)

**On the above line of code we have added matching mobile number column to the apollo data**

In [606]:
matching_mobile_count = dfapollo['mobile'].count()
print("We have got ",matching_mobile_count," matching mobile numbers")

We have got  433  matching mobile numbers


In [608]:
#lets verify if mobile column is added
dfapollo.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name', 'email',
       'email validation', 'job_title', 'company', 'website', 'industry',
       'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about',
       'job posting', 'full links', 'job link', 'sd_team_size', 'mobile'],
      dtype='object')

Now only extra cognism data need to added to complete the cognism data process

In [611]:
# Lets Identify rows in finalcog that do not have a match in dfapollo
non_matching_finalcog = cog_mobile_filter[~cog_mobile_filter['linkedin_link'].isin(dfapollo['linkedin_link'])]

#lets count apollo's no matching contacts
n_m_count = non_matching_finalcog['linkedin_link'].count()
print("We have ",n_m_count," no matching contacts")

# Add a new column 'extra_cognism' to the non-matching rows and set it to True
non_matching_finalcog['extracognism'] = True

# Fill NaN values in the 'extra_cognism' column for rows that came from dfapollo
# result['extra_cognism'] = result['extra_cognism'].fillna(False)


We have  283  no matching contacts


C:\Users\amir\AppData\Local\Temp\ipykernel_20800\3529325103.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matching_finalcog['extracognism'] = True


In [613]:
non_matching_finalcog.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name',
       'email_validation', 'email', 'job_title', 'company', 'website',
       'industry', 'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about', 'mobile',
       'sd_team_size', 'job posting', 'job link', 'full links',
       'extracognism'],
      dtype='object')

In [615]:
print(len(non_matching_finalcog.columns)) #data frame 1
print(len(dfapollo.columns))#data frame 2

23
22


Now we have equal columns in our dfapollo and non_matching_finalcog only need to merge df_apollo and extracognism dataset later

**We will add these dataset together**

Lets create a csv file for the data where we dont able to find mobile numbers

**Step 5 to create a csv file of no mobile number which need to send to zoominfo**

In [621]:
# Fill missing mobile numbers with a default value
dfapollo['mobile'] = dfapollo['mobile'].fillna('No Mobile Number')

In [623]:
# Filter rows without mobile numbers
dfapollo_no_mobile = dfapollo[dfapollo['mobile'].isna() | (dfapollo['mobile'] == 'No Mobile Number')]
#dfapollo_no_mobile

In [625]:
previous_apollo_lenghth = len(dfapollo)

In [627]:
#lets remove no mobile number data from dfapollo dataset so that we can combine both with mobile number dataset later.
# Remove dfapollo_no_mobile from dfapollo (keep only rows with valid mobile numbers)
dfapollo = dfapollo[~(dfapollo['mobile'].isna() | (dfapollo['mobile'] == 'No Mobile Number'))]

In [629]:
#Lets verify if the no mobile contact has been deleted from dfapollo
print(len(dfapollo_no_mobile))
print(len(dfapollo))

654
433


In [631]:
print("Fianl dfapollo length =", previous_apollo_lenghth - len(dfapollo_no_mobile))

Fianl dfapollo length = 433


** Now we need to download a csv file to send to zoominfo**

In [634]:
dfapollo_no_mobile.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name', 'email',
       'email validation', 'job_title', 'company', 'website', 'industry',
       'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about',
       'job posting', 'full links', 'job link', 'sd_team_size', 'mobile'],
      dtype='object')

In [636]:
copy_dfapollo_no_mobile = dfapollo_no_mobile.copy()


In [638]:
# Lets create a copy of the dfapollo_no_mobile dataframe so that we can element unwanted columns and keep only required columns for zoominfo


dfnomobile_col_need_to_delete =['linkedin_link','email validation', 'job_title', 'website', 'industry',
'keywords', 'employees', 'company_city', 'company_state',
'company_country', 'company_linkedin_link', 'company_about',
'job posting', 'full links', 'job link', 'sd_team_size', 'mobile']

#Lets delete the created list from dfapollo_no_mobile data
copy_dfapollo_no_mobile.drop(columns=dfnomobile_col_need_to_delete, inplace=True)

print(dfapollo_no_mobile.columns)
print(copy_dfapollo_no_mobile.columns)

Index(['linkedin_link', 'full_name', 'first_name', 'last_name', 'email',
       'email validation', 'job_title', 'company', 'website', 'industry',
       'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about',
       'job posting', 'full links', 'job link', 'sd_team_size', 'mobile'],
      dtype='object')
Index(['full_name', 'first_name', 'last_name', 'email', 'company'], dtype='object')


In [640]:
import os
# copy_dfapollo_no_mobile.to_csv(os.path.join('C:\\Users\\amir\\OneDrive\\Desktop\\New Hiring lead process\\Using python\\Zoominfo\\send to zoominfo\\',input()))

In [642]:
#lets remove mobile column from the dataset so that we can add mobile column from zoominfo
del dfapollo_no_mobile['mobile']
#why deleting the mobile column
#its because we dont want to have two mobile columns in our data
#one very import thing need to remember that we delete mobile column for the dataset doesnt have mobile numbers in it

In [644]:
dfapollo_no_mobile_len_before = len(dfapollo_no_mobile)
print(len(dfapollo_no_mobile))
print(len(dfapollo_no_mobile.columns))

654
21


**Step 6**

In [647]:
#lets upload the zoominfo data
zoomdata = pd.read_csv("C:\\Users\\amir\\OneDrive\\Desktop\\New Hiring lead process\\Using python\\Zoominfo\\From zoominfo\\from zoominfo 25 march hiring.csv")
len(zoomdata)

304

In [649]:
zoomdata.columns = zoomdata.columns.str.strip().str.lower()

**lets make this zoom data ready to map by renaming and cleaning some data**

In [652]:
#lets remove no mobile number data
zoomdata = zoomdata[zoomdata['mobile phone'].notna() & (zoomdata['mobile phone'] != '')]#removing contact with no mobile
len(zoomdata)

304

In [654]:
#lets rename linkedin column name as our previous data names
zoomdata = zoomdata.rename(columns={'linkedin contact profile url': 'linkedin_link','mobile phone':'mobile'})#column names we already knows otherwise we can check
print("Columns in DataFrame:", zoomdata.columns.tolist())


Columns in DataFrame: ['zoominfo contact id', 'last name', 'first name', 'middle name', 'salutation', 'suffix', 'job title', 'management level', 'job start date', 'job function', 'department', 'company division name', 'direct phone number', 'email address', 'email domain', 'mobile', 'highest level of education', 'contact accuracy score', 'contact accuracy grade', 'zoominfo contact profile url', 'linkedin_link', 'notice provided date', 'person street', 'person city', 'person state', 'person zip code', 'country', 'zoominfo company id', 'company name', 'website', 'founded year', 'company hq phone', 'fax', 'ticker', 'revenue (in 000s usd)', 'revenue range (in usd)', 'employees', 'employee range', 'sic code 1', 'sic code 2', 'sic codes', 'naics code 1', 'naics code 2', 'naics codes', 'primary industry', 'primary sub-industry', 'all industries', 'all sub-industries', 'industry hierarchical category', 'secondary industry hierarchical category', 'alexa rank', 'zoominfo company profile url', 'l

In [656]:
#lets filter UK mobile number data
zoomdata['mobile'] = zoomdata['mobile'].astype(str).str.strip() #formating the mobile numbers as it contain spaces and speacial char
zoom_mobile_filter = zoomdata[zoomdata['mobile'].str.startswith('+44 7')]
len(zoom_mobile_filter)

253

In [658]:
zoom_mobile_filter.columns

Index(['zoominfo contact id', 'last name', 'first name', 'middle name',
       'salutation', 'suffix', 'job title', 'management level',
       'job start date', 'job function', 'department', 'company division name',
       'direct phone number', 'email address', 'email domain', 'mobile',
       'highest level of education', 'contact accuracy score',
       'contact accuracy grade', 'zoominfo contact profile url',
       'linkedin_link', 'notice provided date', 'person street', 'person city',
       'person state', 'person zip code', 'country', 'zoominfo company id',
       'company name', 'website', 'founded year', 'company hq phone', 'fax',
       'ticker', 'revenue (in 000s usd)', 'revenue range (in usd)',
       'employees', 'employee range', 'sic code 1', 'sic code 2', 'sic codes',
       'naics code 1', 'naics code 2', 'naics codes', 'primary industry',
       'primary sub-industry', 'all industries', 'all sub-industries',
       'industry hierarchical category',
       'secondary

In [660]:
# Create a lookup dictionary from zoomdata
lookup_dict = zoom_mobile_filter.set_index('linkedin_link')['mobile'].to_dict()
# Add a new 'mobile' column to dfapollo_no_mobile using the lookup dictionary
dfapollo_no_mobile['mobile'] = dfapollo_no_mobile['linkedin_link'].map(lookup_dict)


**By the above code we have added zoominfo mobile data to our dfapollo_no_mobile data where we dont get mobile numbers from cognism**

**Since we are not using Lusha for mobile number lets remove all the missing mobile number data**

In [663]:
missing_count = dfapollo_no_mobile['mobile'].isna().sum()
missing_count

443

In [665]:
# Fill missing mobile numbers with a default value
dfapollo_no_mobile['mobile'] = dfapollo_no_mobile['mobile'].fillna('No Mobile Number')
#lets remove no mobile number data from dfapollo dataset so that we can combine both with mobile number dataset later.
#keep only rows with valid mobile numbers
dfapollo_no_mobile = dfapollo_no_mobile[~(dfapollo_no_mobile['mobile'].isna() | (dfapollo_no_mobile['mobile'] == 'No Mobile Number'))]

In [669]:
print(dfapollo_no_mobile_len_before - len(dfapollo_no_mobile))
print(missing_count)
print("dfapollo no mobile length after removing invalid or missing mobile numbers",len(dfapollo_no_mobile))

443
443
dfapollo no mobile length after removing invalid or missing mobile numbers 211


If required in future we can keep the missing mobile number data as well

Now we need to add both the data set where we have mobile numbers

**Lets combine all the three dataset together**

In [678]:
dfapollo.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name', 'email',
       'email validation', 'job_title', 'company', 'website', 'industry',
       'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about',
       'job posting', 'full links', 'job link', 'sd_team_size', 'mobile'],
      dtype='object')

In [680]:
non_matching_finalcog.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name',
       'email_validation', 'email', 'job_title', 'company', 'website',
       'industry', 'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about', 'mobile',
       'sd_team_size', 'job posting', 'job link', 'full links',
       'extracognism'],
      dtype='object')

In [682]:
dfapollo_no_mobile.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name', 'email',
       'email validation', 'job_title', 'company', 'website', 'industry',
       'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about',
       'job posting', 'full links', 'job link', 'sd_team_size', 'mobile'],
      dtype='object')

In [684]:
print(len(dfapollo.columns))
print(len(non_matching_finalcog.columns))#non_matching_finalcog has extrac cognims column
print(len(dfapollo_no_mobile.columns))

22
23
22


Lets add all the data set having mobile numbers

In [687]:
# Combine the two DataFrames vertically
master_df = pd.concat([dfapollo, dfapollo_no_mobile, non_matching_finalcog], axis=0, ignore_index=True)
master_df.columns
print(len(master_df.columns))
print(len(master_df))

24
927


In [689]:
#This step should perform at the end of the notebook
#remove bad job title from apollo data
#creating a list of badjobtitles to elemenet bad job titles
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSUQsbmb3MlADPi5q92Eeh5W6nwa8sgszHtctytwl_y2nlUnkJOHj377DJzasaugC_z7yrDU0FKA9cC/pub?gid=0&single=true&output=csv"
badtitle = pd.read_csv(url)
badtitle.columns

Index(['Bad Job Title', 'Unnamed: 1', 'Bad Job Posts'], dtype='object')

In [691]:
bad_title_list = badtitle['Bad Job Title'].tolist()
bad_job_post = badtitle['Bad Job Posts']

master_df = master_df[~master_df['job_title'].isin(bad_title_list)]#deleting the created list from Job title column
master_df = master_df[~master_df['job posting'].isin(bad_job_post)] #deleting the created list from Job Posts column
len(master_df)

886

**From Final data we need to remove the Active sequence data**

In [694]:
active_sequenec = pd.read_csv("C:\\Users\\amir\\OneDrive\\Desktop\\New Hiring lead process\\Using python\\Active Sequene data\\active sequence march 25-025.csv")

In [696]:
active_sequenec.columns

Index(['LinkedIn', 'Name', 'Name - Company', 'Job Title', 'Email', 'Phone',
       'Tags', 'Created At', 'Type - Company', 'Id'],
      dtype='object')

In [698]:
as_need_remove = active_sequenec['LinkedIn'].tolist()
master_df = master_df[~master_df['linkedin_link'].isin(as_need_remove)]#deleting the created list from Job title column
len(master_df)

864

In [700]:
master_df.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name', 'email',
       'email validation', 'job_title', 'company', 'website', 'industry',
       'keywords', 'employees', 'company_city', 'company_state',
       'company_country', 'company_linkedin_link', 'company_about',
       'job posting', 'full links', 'job link', 'sd_team_size', 'mobile',
       'email_validation', 'extracognism'],
      dtype='object')

**Lets add Simplified name**

In [704]:
import re
def simplify_company_name(name):
    if pd.isna(name):
        return name
    
    # Step 1: Remove text within parentheses
    name = re.sub(r'\s*\([^)]*\)', '', str(name))
    
    # Step 2: Remove corporate suffixes (case-insensitive)
    name = re.sub(
        r'(?i)\b(pvt|ltd|inc|co|corp|llc|limited|corporation|company|The)\b', 
        '', 
        name
    )
    
    # Trim whitespace
    name = name.strip()
    
    return name

# Apply to your DataFrame
master_df['simplified_name'] = master_df['company'].apply(simplify_company_name)

In [706]:
master_df= master_df[['linkedin_link','full_name','first_name','last_name','email_validation','email','mobile','job_title','company','simplified_name','website',
                          'industry','keywords','employees','company_city','company_state','company_country','company_linkedin_link','company_about',
                          'job posting', 'job link', 'sd_team_size', 'extracognism']]
master_df.columns

Index(['linkedin_link', 'full_name', 'first_name', 'last_name',
       'email_validation', 'email', 'mobile', 'job_title', 'company',
       'simplified_name', 'website', 'industry', 'keywords', 'employees',
       'company_city', 'company_state', 'company_country',
       'company_linkedin_link', 'company_about', 'job posting', 'job link',
       'sd_team_size', 'extracognism'],
      dtype='object')

In [708]:
master_df.to_csv(os.path.join('C:\\Users\\amir\\OneDrive\\Desktop\\New Hiring lead process\\Using python\\Final python sheet\\',input()))

 onlymobiledata.csv


In [1]:
#master_df.sample(5)